In [ ]:
!pip install numpy opencv-python matplotlib torch torchvision scipy tqdm ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 911.6/911.6 kB 23.3 MB/s eta 0:00:00


In [ ]:
!pip install catboost
!pip install scenedetect
!pip install streamlit -q
!pip install pyngrok
!pip -q install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.3/127.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.9 MB/s eta 0:00:00


In [ ]:
# ngrok API 키
ngrok.set_auth_token('')

# OpenAI API 키를 하드코딩하여 설정
api_key = ""  # 여기에 API 키를 입력하세요

# API 키를 환경 변수에 저장
os.environ['OPENAI_API_KEY'] = api_key

# 확인 메시지 출력
print("API 키가 환경 변수에 성공적으로 설정되었습니다.")


API 키가 환경 변수에 성공적으로 설정되었습니다.


In [ ]:
%%writefile app.py

import os
import sys
import cv2
import subprocess
import numpy as np
import streamlit as st
from pyngrok import ngrok
from PIL import Image, ImageOps

# 모델 실행 함수 -----------------------------------------
def process_video(input_video_path, output_video_path, ball_model, court_model, bounce_model):
    """
    동영상을 처리하는 함수.

    매개변수:
        input_video_path (str): 입력 동영상 파일 경로
        output_video_path (str): 처리된 동영상 출력 경로
        ball_model (str): 공 추적 모델 경로
        court_model (str): 코트 검출 모델 경로
        bounce_model (str): 바운스 검출 모델 경로
    """
    command = [
        "python", "/content/TennisProject/main.py",
        "--path_ball_track_model", "/content/ball_model.pt",
        "--path_court_model", "/content/court_detection.pt",
        "--path_bounce_model", "/content/bounce_detection.cbm",
        "--path_input_video", input_video_path,
        "--path_output_video", "/content/tennis_detection.mp4"
    ]

    # subprocess.run으로 외부 명령어 실행
    result = subprocess.run(
        command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )

    if result.returncode != 0:
        # 실행 실패 시 에러 메시지 반환
        return False, result.stderr
    else:
        # 실행 성공 시 성공 메시지 반환
        return True, result.stdout
# -----------------------------------------
# 프레임 별 이미지 자르는 함수 -----------------------------------------
def extract_frames_from_video(video_path, output_folder='/content/image', frame_rate=1):
    """
    동영상에서 일정 간격으로 프레임을 추출하여 저장하는 함수

    매개변수:
        video_path (str): 동영상 파일 경로
        output_folder (str): 추출된 프레임을 저장할 폴더 경로 (기본값: '/content/image')
        frame_rate (int): 초당 추출할 프레임 수 (기본값: 1초당 1프레임)
    """
    # 동영상 파일 로드
    cap = cv2.VideoCapture(video_path)

    # 동영상 파일이 열렸는지 확인
    if not cap.isOpened():
        print("오류: 동영상을 열 수 없습니다.")
        return

    # 저장할 폴더가 없으면 생성
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"폴더 생성됨: {output_folder}")

    # 동영상의 FPS(초당 프레임 수) 가져오기
    fps = cap.get(cv2.CAP_PROP_FPS)  # FPS: Frames Per Second
    frame_interval = int(fps / frame_rate)  # 추출할 프레임 간격 계산

    frame_count = 0  # 현재 프레임 번호
    saved_frame_count = 0  # 저장된 프레임 수

    while True:
        # 동영상에서 다음 프레임 읽기
        ret, frame = cap.read()

        # 더 이상 프레임이 없으면 반복 종료
        if not ret:
            break

        # 지정된 간격에 해당하는 프레임만 저장
        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f'image{saved_frame_count:04d}.jpg')  # 파일 이름 생성
            cv2.imwrite(frame_filename, frame)  # 프레임 저장
            print(f"저장됨: {frame_filename}")  # 저장 완료 메시지 출력
            saved_frame_count += 1  # 저장된 프레임 수 증가

        # 현재 프레임 번호 증가
        frame_count += 1

    # 동영상 파일 닫기
    cap.release()
    # 모든 OpenCV 창 닫기
    cv2.destroyAllWindows()
# -----------------------------------------

st.title("테니스 영상 해설")
file = st.file_uploader("영상을 업로드하세요.", type=['mp4', 'avi', 'mov'])

if file is None:
    st.text("동영상을 먼저 업로드해주세요.")
else:
    # 동영상 저장 경로
    input_video_path = "/content/tennis.mp4"
    with open(input_video_path, "wb") as f:
        f.write(file.read())
    st.success("동영상이 성공적으로 업로드되었습니다!")

    # 명령어 실행
    st.text("동영상을 처리 중입니다...")
    input_video_path = "/content/tennis.mp4"
    output_video_path = "/content/tennis_detection.mp4"
    ball_model_path = "/content/ball_model.pt"
    court_model_path = "/content/court_detection.pt"
    bounce_model_path = "/content/bounce_detection.cbm"

    # 동영상 처리 함수 호출
    success, message = process_video(
        input_video_path=input_video_path,
        output_video_path=output_video_path,
        ball_model=ball_model_path,
        court_model=court_model_path,
        bounce_model=bounce_model_path
    )

    if success:
        st.success("동영상 처리가 완료되었습니다!")
        st.text(message)
    else:
        st.error(f"오류: 동영상 처리 중 문제가 발생했습니다.\n{message}")

    # 동영상 파일 경로 설정
    video_path = '/content/tennis_detection.mp4'  # 처리된 동영상 경로
    output_folder = 'image'  # 저장할 폴더 이름

    # 프레임 별 이미지 함수 호출
    extract_frames_from_video(video_path, output_folder, frame_rate=1)

    # 추출된 첫 번째 프레임 경로
    first_frame_path = os.path.join(output_folder, '/content/image/image0000.jpg')

    if os.path.exists(first_frame_path):
        # 첫 번째 프레임을 열어서 화면에 표시
        st.text("동영상 처리가 완료되었습니다!")
        image = Image.open(first_frame_path)
        st.image(image, use_container_width=True)
    else:
        # 프레임 파일이 없을 때 에러 메시지 출력
        st.error("첫 번째 프레임이 존재하지 않습니다. 프레임 추출 과정을 확인하세요.")

Writing app.py


In [ ]:
!ls

app.py	ball_model.pt  bounce_detection.cbm  court_detection.pt  sample_data


In [ ]:
!nohup streamlit run app.py --server.port 80 &

nohup: appending output to 'nohup.out'


In [ ]:
!git clone https://github.com/yastrebksv/TennisProject.git
%cd TennisProject

Cloning into 'TennisProject'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 45 (delta 8), reused 7 (delta 7), pack-reused 30 (from 1)
Receiving objects: 100% (45/45), 5.50 MiB | 9.07 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/content/TennisProject


In [ ]:
import os
os.chdir('/content')

In [ ]:
url = ngrok.connect(addr=80)  # 80번 포트를 노출
url

<NgrokTunnel: "https://d717-34-125-130-189.ngrok-free.app" -> "http://localhost:80">

In [ ]:
ngrok.kill()

NameError: name 'ngrok' is not defined